##### Copyright 2020 Google

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# QAOA Tasks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.example.org/cirq/research/qaoa/tasks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on QuantumLib</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/ReCirq/blob/docs/docs/qaoa/tasks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/ReCirq/blob/docs/docs/qaoa/tasks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/ReCirq/docs/qaoa/tasks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Setup

Install the ReCirq package:

In [ ]:
!pip install git+https://github.com/quantumlib/ReCirq

 Install Cirq's `pytket` interface:

In [ ]:
!pip install pytket-cirq

And import ReCirq:

In [ ]:
import recirq

## Problem generation

First, we generate and save all of the random instances of the problem. This is not computationally intensive but very important to do first so we have a fixed set of random instances.

In [ ]:
from recirq.qaoa.experiments.problem_generation_tasks import \
    SKProblemGenerationTask, HardwareGridProblemGenerationTask, ThreeRegularProblemGenerationTask, \
    generate_3_regular_problem, generate_sk_problem, generate_hardware_grid_problem

pgen_dataset_id = '2020-03-tutorial'
hardware_grid_problem_tasks = [
    HardwareGridProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        device_name='Sycamore23',
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(2, 8 + 1, 2)
]
recirq.display_markdown_docstring(HardwareGridProblemGenerationTask)

In [ ]:
sk_problem_tasks = [
    SKProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(3, 7 + 1, 2)
]
recirq.display_markdown_docstring(SKProblemGenerationTask)

In [ ]:
three_regular_problem_tasks = [
    ThreeRegularProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(3, 8 + 1) if 3 * n % 2 == 0
]

### Run the tasks

In [ ]:
for task in hardware_grid_problem_tasks:
    generate_hardware_grid_problem(task)
for task in sk_problem_tasks:
    generate_sk_problem(task)
for task in three_regular_problem_tasks:
    generate_3_regular_problem(task)

## Angle precomputation

In [ ]:
from recirq.qaoa.experiments.angle_precomputation_tasks import \
    AnglePrecomputationTask, precompute_angles

apre_dataset_id = '2020-03-tutorial'
precompute_tasks = [
    AnglePrecomputationTask(
        dataset_id=apre_dataset_id,
        generation_task=gen_task,
        p=p)
    for gen_task in recirq.roundrobin(
        hardware_grid_problem_tasks,
        sk_problem_tasks,
        three_regular_problem_tasks,
    )
    for p in range(1, 3 + 1)
]
recirq.display_markdown_docstring(AnglePrecomputationTask)

In [ ]:
for task in precompute_tasks:
    precompute_angles(task)

## Precomputed angle data collection

In [ ]:
from recirq.qaoa.experiments.precomputed_execution_tasks import \
    PrecomputedDataCollectionTask, collect_data
    

dcol_dataset_id = '2020-03-tutorial'
data_collection_tasks = [
    PrecomputedDataCollectionTask(
        dataset_id=dcol_dataset_id,
        precomputation_task=pre_task,
        device_name='Syc23-simulator',
        n_shots=50_000,
        structured=True,
    )
    for pre_task in precompute_tasks
]
recirq.display_markdown_docstring(PrecomputedDataCollectionTask)

In [ ]:
await recirq.execute_in_queue(collect_data, data_collection_tasks, num_workers=2)

## Landscape data collection

In [ ]:
from recirq.qaoa.experiments.p1_landscape_tasks import \
    P1LandscapeDataCollectionTask, \
    get_data_collection_tasks_on_a_grid, \
    collect_either_landscape_or_cal

recirq.display_markdown_docstring(P1LandscapeDataCollectionTask)

In [ ]:
hardware_grid_problem_task = HardwareGridProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    device_name='Sycamore23',
    instance_i=0,
    n_qubits=4
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=hardware_grid_problem_task,
    dataset_id=dcol_dataset_id,
    gamma_res=11,
    beta_res=11,
    device_name='Syc23-simulator',
    epoch="grid")

await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)

In [ ]:
sk_problem_task = SKProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    instance_i=0,
    n_qubits=3,
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=sk_problem_task,
    dataset_id=dcol_dataset_id,
    gamma_res=11,
    beta_res=11,
    device_name='Syc23-simulator',
    epoch="sk")
await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)

In [ ]:
three_regular_problem_task = ThreeRegularProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    instance_i=0,
    n_qubits=4
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=three_regular_problem_task,
    dataset_id=dcol_dataset_id,
    device_name='Syc23-simulator',
    gamma_res=11,
    beta_res=11,
    epoch="tr")

await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)